In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('res.csv', sep=',').astype(np.float64)
df

In [ ]:
df_new = df[df.sample_probability==0.3]
df_new

In [ ]:
df_new.plot(x='num_qubits',y=['time_ratio_without_swap_opt', 'time_ratio_with_swap_opt'], logy=True)
df_new.plot(x='num_qubits',y=['cx_count_ratio_without_swap_opt', 'cx_count_ratio_with_swap_opt'])

In [ ]:
df_new.plot(x='num_qubits',y=['expected_fidelity_baseline', 'expected_fidelity_new_scheme_with_swap_opt', 'expected_fidelity_new_scheme_without_swap_opt'], logy=True)